# Deploy RoBERTa Embedding Model Package from AWS Marketplace 


This sample notebook shows you how to deploy [RoBERTa Embedding](https://aws.amazon.com/marketplace/pp/prodview-nvrw726ubwj26?sr=0-2&ref_=beagle&applicationId=AWSMPContessa) using Amazon SageMaker. You could use the embedding for semantic similarity and building RAG solutions.


## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to  [RoBERTa Embedding](https://aws.amazon.com/marketplace/pp/prodview-nvrw726ubwj26?sr=0-2&ref_=beagle&applicationId=AWSMPContessa). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page:[RoBERTa Embedding](https://aws.amazon.com/marketplace/pp/prodview-nvrw726ubwj26?sr=0-2&ref_=beagle&applicationId=AWSMPContessa).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn = "arn:aws:sagemaker:us-east-1:865070037744:model-package/robertembeddingmodel-b89cf0d12461354199f7436958cd2800"

In [2]:
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/08/25 01:48:21] INFO     Found credentials from IAM Role:                                   ]8;id=625527;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=231707;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "roberta-embedding"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.m4.xlarge"
)
batch_transform_inference_instance_type = (
    "ml.m4.xlarge"
)

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

[01/08/25 01:48:34] INFO     Creating model with name:                                              ]8;id=300488;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=91466;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             robertembeddingmodel-b89cf0d12461354199-2025-01-08-01-48-34-881                       

[01/08/25 01:48:35] INFO     Creating endpoint-config with name roberta-embedding                   ]8;id=183046;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=49541;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name roberta-embedding                          ]8;id=730479;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=389818;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [6]:
expected_input_format = '{"sentences": ["Hello, how are you doing?", "I am doing great, how about you?"]}'
file_name = "input.json"
with open(file_name, "w") as f:
    f.write(expected_input_format)

<Add code snippet that shows the payload contents>

In [7]:
output_file_name = "output.json"

### C. Perform real-time inference

In [8]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type $content_type --region $sagemaker_session.boto_region_name $output_file_name

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


### D. Visualize output

In [9]:
!cat output.json

{"embeddings":[[-0.0666792094707489,0.19491320848464966,0.038498807698488235,-0.04005127772688866,0.2634272575378418,-0.08368854969739914,0.11327042430639267,0.02045675367116928,0.10610690712928772,0.1880166083574295,-0.17091697454452515,-0.09246671199798584,0.02027946151793003,-0.15892991423606873,0.06262950599193573,0.031094400212168694,-0.009863250888884068,-0.1775425523519516,-0.12298436462879181,0.07201690226793289,-0.12650197744369507,0.14150585234165192,-0.07261771708726883,0.05795624479651451,0.07602754980325699,0.05466923862695694,0.008161429315805435,0.006116125266999006,0.007487546186894178,5.144253373146057e-05,-0.03382505103945732,-0.13423773646354675,0.07898472994565964,-0.09441737830638885,-0.12909917533397675,0.0117171136662364,0.029190074652433395,0.03510036692023277,0.435072124004364,0.13692380487918854,0.13320361077785492,-0.3388993442058563,-0.04491126909852028,0.0023823296651244164,0.03150381147861481,-0.06493967026472092,-0.08576779812574387,-0.12210739403963089,0

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [10]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

[01/08/25 01:59:15] INFO     Deleting endpoint with name: roberta-embedding                         ]8;id=75090;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=273280;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\

                    INFO     Deleting endpoint configuration with name: roberta-embedding           ]8;id=703446;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=187891;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4865\4865]8;;\

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [12]:
ls -la  data/input/batch/

total 12
drwxrwxr-x 2 ec2-user ec2-user 4096 Jan  6 23:35 ./
drwxrwxr-x 4 ec2-user ec2-user 4096 Jan  6 22:17 ../
-rw-rw-r-- 1 ec2-user ec2-user  193 Jan  6 23:21 input.jsonl


In [13]:
# upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-1-809103429415/roberta-embedding


In [14]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type, strategy='SingleRecord')
transformer.transform(transform_input, content_type=content_type, split_type='Line')
transformer.wait()

[01/08/25 01:59:57] INFO     Creating model with name:                                              ]8;id=432713;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=636486;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             robertembeddingmodel-b89cf0d12461354199-2025-01-08-01-59-57-448                       

                    INFO     Creating transform job with name:                                      ]8;id=132761;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=981842;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#3951\3951]8;;\
                             robertembeddingmodel-b89cf0d12461354199-2025-01-08-01-59-57-991                       

..............................................Starting the inference server with 4 workers.
[2025-01-08 02:07:38 +0000] [10] [INFO] Starting gunicorn 23.0.0
[2025-01-08 02:07:38 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2025-01-08 02:07:38 +0000] [10] [INFO] Using worker: sync
[2025-01-08 02:07:38 +0000] [12] [INFO] Booting worker with pid: 12
[2025-01-08 02:07:38 +0000] [13] [INFO] Booting worker with pid: 13
[2025-01-08 02:07:38 +0000] [14] [INFO] Booting worker with pid: 14
[2025-01-08 02:07:38 +0000] [15] [INFO] Booting worker with pid: 15
INFO:embedding_service:Model and tokenizer loaded successfully on device: cpu
INFO:embedding_service:Model and tokenizer loaded successfully on device: cpu
INFO:embedding_service:Model and tokenizer loaded successfully on device: cpu
INFO:embedding_service:Model and tokenizer loaded successfully on device: cpu
169.254.255.130 - - [08/Jan/2025:02:07:46 +0000] "GET /ping HTTP/1.1" 200 2 "-" "Go-http-client/1.1"
169.254.255.130 

In [ ]:
# output is available on following path
transformer.output_path

The output path returned above should contain a file "input.jsonl.out" (input file name suffixed with .out). The output in this file must match the output available in data/output/batch folder.

## 4. Clean-up

### A. Delete the model

In [16]:
model.delete_model()

[01/08/25 02:08:32] INFO     Deleting model with name:                                              ]8;id=167784;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=977295;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5226\5226]8;;\
                             robertembeddingmodel-b89cf0d12461354199-2025-01-08-01-59-57-448                       

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

